In [ ]:
# mainly from https://github.com/borutb-fri/FMLD/blob/main/mask-test.py
# Download model from:
# https://unilj-my.sharepoint.com/:u:/g/personal/borutb_fri1_uni-lj_si/EdmDsIgG9XBJkRVXDKyOwvEBK7pK1EEq9cBfOVm3kLzPvw?e=M9pULa
# model also from https://github.com/borutb-fri/FMLD/

import os
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch import nn

# Applying Transforms to the Data
image_transforms = { 
    'test': transforms.Compose([
        transforms.Resize(size=(224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

# Load the Data
dataset = 'DS'
test_directory = os.path.join(dataset, 'test')

# Batch size
bs = 128

# Number of classes
num_classes = 2

# Load Data from folders
data = {
    'test': datasets.ImageFolder(root=test_directory, transform=image_transforms['test']),
}

class_names = data['test'].classes
transform=image_transforms['test']


# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['test'].class_to_idx.items()}
print('Classes: ',idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
test_data_size = len(data['test'])

# Create iterators for the Data loaded using DataLoader module
test_data_loader = DataLoader(data['test'], batch_size=bs, shuffle=False)

# Print the test set data sizes
print('Number of faces: ',test_data_size)

def computeTestSetAccuracy(model, loss_criterion, data_loader, data_size):
    '''
    Function to compute the accuracy on the test set
    Parameters
        :param model: Model to test
        :param loss_criterion: Loss Criterion to minimize
    '''

    test_acc = 0.0
    test_loss = 0.0

    # Validation - No gradient tracking needed
    with torch.no_grad():
        # Set to evaluation mode
        model.eval()

        # Validation loop
        for j, (inputs, labels) in enumerate(data_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            #loss = loss_criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            #test_loss += loss.item() * inputs.size(0)

            # Calculate validation accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
        
            print(acc)
            # Compute total accuracy in the whole batch and add to valid_acc
            test_acc += acc.item() * inputs.size(0)

            
    # Find average test loss and test accuracy
    #avg_test_loss = test_loss/data_size
    avg_test_acc = test_acc/data_size
    return avg_test_acc



device = torch.device('cpu')
loss_func = nn.CrossEntropyLoss() #for a multi-class classification problem 
  
model_file = 'resnet152.pt'
if os.path.exists(model_file):    
    model = torch.load(model_file, map_location=device)
    model = model.to(device)
    avg_test_acc=computeTestSetAccuracy(model, loss_func, test_data_loader, test_data_size)
    print("Test accuracy : " + str(avg_test_acc))
else:
    print("Warrning: No Pytorch model for classification: resnet152.pt. Please Download it from GitHub link.\n")

Classes:  {0: 'correct', 1: 'incorrect'}
Number of faces:  6995


/home/makkusu/.conda/envs/main/lib/python3.9/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/makkusu/.conda/envs/main/lib/python3.9/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/makkusu/.conda/envs/main/lib/python3.9/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original s

tensor(0.9688)
tensor(0.9922)
tensor(0.9688)
tensor(0.9141)
tensor(0.9297)
tensor(0.8750)
